# BTS, Focused on LGA

Edit selector= below

Look at the beacons with the largest normalized spread.

In [1]:
import math
import pandas as pd
import BQhelper as bq
import matplotlib.pyplot as plt

bq.project = "mlab-sandbox"
# bq.dataset = 'mattmathis'
# bq.UnitTestRunQuery()
# bq.UnitTestWriteQuery()


In [2]:
query="""
SELECT
  a.TestTime,
  client.IP,
  a.MeanThroughputMbps,
  node._instruments
# FROM   `mlab-sandbox.mm_unified_testing.unified_downloads`
FROM `measurement-lab.ndt.unified_downloads`
WHERE client.IP in ( {clientIP} )
AND test_date > '2020-06-01'
ORDER BY TestTime

"""

In [3]:
%matplotlib nbagg
# from matplotlib import interactive
# interactive(True)
plt.ion()

clients = [
    '69.68.23.44', # Max deltaMean
    '96.229.66.58' # Max deltaMax
]


def plotMultiBeacons(clients, columns=1, width=10, data=None, selector='downloads'):
    if data is None:
        clist = '"'+'", "'.join(clients)+'"'
        data=bq.QueryTimestampTimeseries(query, clientIP=clist, selector=selector)
        global StashData # Skip slow queries when debugging
        StashData = data
    rows = math.ceil(len(clients) / float(columns))
    figLen = width/float(columns)*rows  # assume square subplots
    print('Size', figLen, width)
    plt.rcParams['figure.figsize'] = [ width, figLen]
    
    fig, axs = plt.subplots(nrows=rows, ncols=columns, squeeze=False, sharex='all')
    for ax, client in zip([i for j in axs for i in j], clients):
        print ('Beacon: '+client)
        ax.set_title('Beacon: '+client)
        cdata = data[data['IP'] == client]
        ax.plot(cdata['MeanThroughputMbps'][cdata["_instruments"] == 'web100'], 'b.',
            cdata['MeanThroughputMbps'][cdata["_instruments"] == 'tcpinfo'], 'r.')
    fig.autofmt_xdate()
    fig.show()
    
    
# plotMultiBeacons(clients, 2, width=10)

In [45]:
# NB: ThousandRandomBeacons4perDay are all domotz tests.

clientQ="""
With Clients AS (
    SELECT ClientIP,
    COUNT (DISTINCT REGEXP_EXTRACT(ParseInfo.TaskFileName, 'mlab[1-4]-[a-z][a-z][a-z][0-9][0-9t]') ) as Servers,
    count(*) AS tests
    FROM `mlab-oti.base_tables.ndt5`
JOIN `mlab-sandbox.mattmathis.ThousandRandomBeacons4perDay` ON ( clientIP = result.C2S.ClientIP )
WHERE
  REGEXP_EXTRACT(ParseInfo.TaskFileName, 'mlab[1-4]-([a-z][a-z][a-z])[0-9][0-9t]') = 'lga'
  AND DATE(TIMESTAMP_SECONDS(log_time)) > '2020-06-05'
  GROUP BY clientIP
)

# Choose
# SELECT * FROM Beacons ORDER BY {selector} LIMIT 30
SELECT * FROM Clients
"""

selector="delta desc"
selector="deltaMax desc"
BeaconIPs=bq.DataFrameQuery(clientQ, selector=selector)

In [47]:
print (BeaconIPs)

           ClientIP  Servers  tests
0      24.63.141.72       11     16
1     66.108.141.64       13     16
2    24.146.180.169        8     16
3   173.220.122.246       12     17
4      24.2.193.185       10     16
5      73.253.3.118       11     15
6      73.61.225.27        9     16
7     73.38.153.126       10     15
8      96.81.116.17       11     16
9    64.119.129.162       10     16
10    47.20.134.163       10     16
11      66.30.43.84       12     15
12   173.77.251.162        8     14
13      73.159.2.78        9     16
14   73.114.215.200        9     16
15     107.3.77.108       10     15
16     47.23.42.146        9     16
17     72.80.56.180       12     16
18    24.61.217.179        8     12
19   157.130.249.98       11     16
20   75.144.150.182        8     16
21    73.182.187.63       10     16
22    73.17.110.183        9     16
23    64.222.170.58        7     15
24    74.14.248.250        6      8
25   76.119.164.158        9     16
26       73.4.54.41        9

In [52]:
# NB: ThousandRandomBeacons4perDay are all domotz tests.

listq="""
With Beacons AS (
SELECT
#  REGEXP_EXTRACT(ParseInfo.TaskFileName, 'mlab[1-4]-[a-z][a-z][a-z][0-9][0-9t]') AS shortname,
  CONCAT(REGEXP_EXTRACT(ParseInfo.TaskFileName, 'mlab[1-4]-([a-z][a-z][a-z][0-9][0-9t])'), '-',
    REGEXP_EXTRACT(ParseInfo.TaskFileName, '(mlab[1-4])-[a-z][a-z][a-z][0-9][0-9t]')) AS revName,
  `mlab-sandbox.mattmathis.Binify`( result.C2S.MeanThroughputMbps, 5) AS rateBin,
FROM `mlab-oti.base_tables.ndt5`
JOIN `mlab-sandbox.mattmathis.ThousandRandomBeacons4perDay` ON ( clientIP = result.C2S.ClientIP )
WHERE
  REGEXP_EXTRACT(ParseInfo.TaskFileName, 'mlab[1-4]-([a-z][a-z][a-z])[0-9][0-9t]') = 'lga'
  AND DATE(TIMESTAMP_SECONDS(log_time)) > '2020-06-05'
  GROUP BY revName, rateBin
)

# Choose
SELECT * FROM Beacons
"""

RateBins=bq.DataFrameQuery(listq)

In [53]:
print (RateBins)

         revName    rateBin
0    lga05-mlab2   46.41589
1    lga03-mlab3   46.41589
2    lga06-mlab1   46.41589
3    lga04-mlab1  215.44347
4    lga05-mlab2  215.44347
..           ...        ...
100  lga05-mlab2  100.00000
101  lga04-mlab3   10.00000
102  lga08-mlab2  215.44347
103  lga04-mlab1   21.54435
104  lga08-mlab1   46.41589

[105 rows x 2 columns]


In [65]:
# NB: ThousandRandomBeacons4perDay are all domotz tests.

SpreadQ="""
With Singletons AS (
SELECT
    COUNT (*) as tests,
    result.S2C.ClientIP,
    REGEXP_EXTRACT(ParseInfo.TaskFileName, 'mlab[1-4]-[a-z][a-z][a-z][0-9][0-9t]') AS shortname,
    `mlab-sandbox.mattmathis.Binify`( result.C2S.MeanThroughputMbps, 5) AS rateBin,
FROM `mlab-oti.base_tables.ndt5`
WHERE
    DATE(TIMESTAMP_SECONDS(log_time)) > '2020-06-05'
    AND REGEXP_EXTRACT(ParseInfo.TaskFileName, 'mlab[1-4]-([a-z][a-z][a-z])[0-9][0-9t]') = 'lga'
    AND result.S2C is not null
GROUP BY
    result.S2C.ClientIP
HAVING
    tests = 1
)

# Choose
SELECT * FROM Singletons
"""

Singeltons=bq.DataFrameQuery(SpreadQ)

print (Singeltons)

400 SELECT list expression references ParseInfo.TaskFileName which is neither grouped nor aggregated at [6:20]

(job ID: ff1ceb73-5f98-408e-a0b7-3b108fed1341)

                                   -----Query Job SQL Follows-----                                   

    |    .    |    .    |    .    |    .    |    .    |    .    |    .    |    .    |    .    |
   1:
   2:With Singletons AS (
   3:SELECT
   4:    COUNT (*) as tests,
   5:    result.S2C.ClientIP,
   6:    REGEXP_EXTRACT(ParseInfo.TaskFileName, 'mlab[1-4]-[a-z][a-z][a-z][0-9][0-9t]') AS shortname,
   7:    `mlab-sandbox.mattmathis.Binify`( result.C2S.MeanThroughputMbps, 5) AS rateBin,
   8:FROM `mlab-oti.base_tables.ndt5`
   9:WHERE
  10:    DATE(TIMESTAMP_SECONDS(log_time)) > '2020-06-05'
  11:    AND REGEXP_EXTRACT(ParseInfo.TaskFileName, 'mlab[1-4]-([a-z][a-z][a-z])[0-9][0-9t]') = 'lga'
  12:    AND result.S2C is not null
  13:GROUP BY
  14:    result.S2C.ClientIP
  15:HAVING
  16:    tests = 1
  17:)
  18:
  19:# Choose
 

BadRequest: 400 SELECT list expression references ParseInfo.TaskFileName which is neither grouped nor aggregated at [6:20]

(job ID: ff1ceb73-5f98-408e-a0b7-3b108fed1341)

                                   -----Query Job SQL Follows-----                                   

    |    .    |    .    |    .    |    .    |    .    |    .    |    .    |    .    |    .    |
   1:
   2:With Singletons AS (
   3:SELECT
   4:    COUNT (*) as tests,
   5:    result.S2C.ClientIP,
   6:    REGEXP_EXTRACT(ParseInfo.TaskFileName, 'mlab[1-4]-[a-z][a-z][a-z][0-9][0-9t]') AS shortname,
   7:    `mlab-sandbox.mattmathis.Binify`( result.C2S.MeanThroughputMbps, 5) AS rateBin,
   8:FROM `mlab-oti.base_tables.ndt5`
   9:WHERE
  10:    DATE(TIMESTAMP_SECONDS(log_time)) > '2020-06-05'
  11:    AND REGEXP_EXTRACT(ParseInfo.TaskFileName, 'mlab[1-4]-([a-z][a-z][a-z])[0-9][0-9t]') = 'lga'
  12:    AND result.S2C is not null
  13:GROUP BY
  14:    result.S2C.ClientIP
  15:HAVING
  16:    tests = 1
  17:)
  18:
  1

In [43]:
# lame
beacons = []
for t in stepbeacons['NDT5']:
    beacons.append(t['clientIP'])
plotMultiBeacons(beacons, 2, width=10)

KeyError: 'NDT5'

In [7]:
# Particularly ugly, 
# plotMultiBeacons(['67.161.83.114'], 1, width=10)
